### Evaluation -> RAG Triad + Answer Correctness
#### For -> Mistral and FAISS (Embedding model used: sentence-transformers/all-MiniLM-L6-v2)
#### Poojitha


In [3]:
! pip install spacy

In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.9 MB/s eta 0:00:00


In [5]:
import spacy
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import string

# Load spaCy model for NLP tasks
nlp = spacy.load("en_core_web_sm")

# Load model and tokenizer
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [6]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/mistral_output.csv')
df

,question,ground_truth,rag_answer,context,possible_questions
0,What are the keywords mentioned in the paper t...,The keywords written in the paper are: Earthqu...,What are the keywords mentioned in the paper t...,NaN,[]
1,What is the purpose of using Artificial Neural...,Artificial Neural Networks (ANNs) are used to ...,What is the purpose of using Artificial Neural...,NaN,[]
2,How does the frequency of a building's natural...,A building's response to an earthquake is dyna...,How does the frequency of a building's natural...,NaN,[]
3,What kind of neural network model is most freq...,The most frequently applied neural network mod...,What kind of neural network model is most freq...,NaN,[]
4,How was the training of the ANN model conducte...,The ANN model was trained using real earthquak...,How was the training of the ANN model conducte...,NaN,[]
5,What is the significance of the study's findin...,The study's findings demonstrate the ability o...,What is the significance of the study's findin...,NaN,[]


In [10]:
# Function to calculate the embedding of a given text
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    # Reshape the tensor to remove the batch dimension and detach it from the computation graph
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().squeeze()

# Function to calculate cosine similarity between two embeddings
def calculate_similarity(embedding1, embedding2):
    # Ensure the embeddings are 2D
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

# Ccalculation for the advanced NLP tasks using spaCy
def calculate_context_relevancy(row):
    doc = nlp(row['context'])
    question_doc = nlp(row['question'])
    question_keywords = {token.lemma_.lower() for token in question_doc if not token.is_stop}
    relevant_sentences = sum(any(token.lemma_.lower() in question_keywords for token in sent) for sent in doc.sents)
    return relevant_sentences / len(list(doc.sents))

# Function to calculate answer relevancy
def calculate_answer_relevancy(row):
    original_question_embedding = get_embedding(row['question'])

    # 'possible_questions' is a list of artificially generated questions from the answer (reverse engineered)
    artificial_questions_embeddings = [get_embedding(q) for q in row['possible_questions']]

    relevancies = [
        calculate_similarity(original_question_embedding, q_emb)
        for q_emb in artificial_questions_embeddings
    ]

    # Calculate the mean of these relevancies to get the answer relevancy score
    answer_relevancy = np.mean(relevancies)
    return answer_relevancy

In [11]:
! pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [12]:
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

# Initialize the text generation pipeline with the specified model
def initialize_pipeline():
    return pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")

# Function to generate responses using the pipeline
def run_mistral_pipeline(question, pipe):
    # Generate response with specific settings including beam search and truncation
    response = pipe(question, max_length=100, num_beams=5, length_penalty=2.0, truncation=True)[0]['generated_text']
    return response

# Initialize embedding model for calculating embeddings and similarities
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embedding(text):
    """Get text embeddings."""
    return model.encode(text, convert_to_tensor=True)

def calculate_similarity(embedding1, embedding2):
    """Calculate cosine similarity between two embeddings."""
    return util.pytorch_cos_sim(embedding1, embedding2).item()

# Define a function to evaluate various relevancies and correctness
def evaluate_relevancies(df):
    df['Context Relevance'] = df.apply(lambda row: calculate_similarity(get_embedding(row['context']), get_embedding(row['rag_answer'])), axis=1)
    df['Answer Relevance'] = df.apply(lambda row: calculate_similarity(get_embedding(row['question']), get_embedding(row['rag_answer'])), axis=1)
    df['Groundedness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['context']), get_embedding(row['rag_answer'])), axis=1)
    df['Answer Correctness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['ground_truth']), get_embedding(row['rag_answer'])), axis=1)
    return df

# Main function to process questions and generate dataset
def main():
    pipe = initialize_pipeline()

    # List of questions to process
    questions = [
        "What are the keywords mentioned in the paper titled Response Prediction of Structural System Subject to Earthquake Motions using Artificial Neural Network?",
        "What is the purpose of using Artificial Neural Networks in earthquake response prediction?",
        "How does the frequency of a building's natural frequency affect its response to an earthquake?",
        "What kind of neural network model is most frequently applied for modeling dynamic response of structures?",
        "How was the training of the ANN model conducted for predicting responses to various intensity earthquakes?",
        "What is the significance of the study's findings on predicting the safeness of structural systems?"
    ]

    # Generate responses for each question using the initialized pipeline
    rag_answers = [run_mistral_pipeline(question, pipe) for question in questions]

    # Mock context and ground truths for demonstration purposes
    contexts = ["Context related to " + question for question in questions]
    ground_truths = ["Expected answer for " + question for question in questions]

    # Create a DataFrame with all data
    data = {
        "question": questions,
        "context": contexts,
        "rag_answer": rag_answers,
        "ground_truth": ground_truths
    }
    df = pd.DataFrame(data)

    # Evaluate the responses
    df = evaluate_relevancies(df)
    print(df)

# Run the main function
if __name__ == "__main__":
    main()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


                                            question  \
0  What are the keywords mentioned in the paper t...   
1  What is the purpose of using Artificial Neural...   
2  How does the frequency of a building's natural...   
3  What kind of neural network model is most freq...   
4  How was the training of the ANN model conducte...   
5  What is the significance of the study's findin...   

                                             context  \
0  Context related to What are the keywords menti...   
1  Context related to What is the purpose of usin...   
2  Context related to How does the frequency of a...   
3  Context related to What kind of neural network...   
4  Context related to How was the training of the...   
5  Context related to What is the significance of...   

                                          rag_answer  \
0  What are the keywords mentioned in the paper t...   
1  What is the purpose of using Artificial Neural...   
2  How does the frequency of a building's natu